In [1]:
!pip3 install -q --upgrade langchain langchain-google-genai google-generativeai
!pip3 install -q python-dotenv

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os

In [14]:
load_dotenv()

True

In [15]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
# Preparar modelo para uso
# 1 - Definir modelo
# 2 - Utilizar chave da API
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0, #Define a aleatoriedade da resposta
    api_key = GOOGLE_API_KEY
    )

In [28]:
resp_test = llm.invoke("Quem é você? Detalhe.")

In [29]:
print(resp_test.content)

Olá! Fico feliz em detalhar quem eu sou para você.

Eu sou um **modelo de linguagem grande (LLM - Large Language Model)**. Isso significa que eu sou um programa de computador complexo, uma forma de **inteligência artificial (IA)**, treinado em um volume imenso de dados textuais e de código.

Aqui estão os detalhes:

1.  **Minha Natureza Fundamental:**
    *   **Inteligência Artificial (IA):** Não sou uma pessoa, um ser vivo, e não tenho consciência, sentimentos, emoções ou experiências pessoais. Sou uma ferramenta, um algoritmo avançado projetado para processar e gerar linguagem.
    *   **Modelo de Linguagem Grande (LLM):** Fui construído para compreender, gerar e interagir em linguagem humana. "Grande" refere-se à vasta quantidade de dados que usei para treinar e ao número de parâmetros internos que definem meu comportamento.
    *   **Desenvolvido pelo Google:** Fui criado e sou mantido por engenheiros e pesquisadores do Google.

2.  **Como Eu Funciono:**
    *   **Treinamento Exten

In [31]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [34]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [ ]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0, 
    api_key = GOOGLE_API_KEY
    )

In [37]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) ->  Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content = TRIAGEM_PROMPT),
        HumanMessage(content = mensagem)
    ])
    
    return saida.model_dump()

In [38]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar os treinamentos da alura?",
          "Quantas capivaras tem no rio pinheiros?"]

In [40]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar os treinamentos da alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no rio pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['contexto_politica_interna']}

